# Tide Prediction Task

Start with some imports.

In [1]:
from gbdxtools import Interface
from shapely.wkt import loads
import dateutil.parser
from datetime import datetime

Create a GBDX interface.

In [2]:
gbdx = Interface()

Start with a catalog ID (this is some image of the Palm Islands in Dubai).

In [3]:
cat_id = '103001000349F800'

Get metadata record for parsing of footprint WKT and timestamp.

In [4]:
record = gbdx.catalog.get(cat_id)

Get the centroid of the footprint.

In [5]:
centroid = loads(record.get('properties').get('footprintWkt')).centroid

The latitude corresponds to `y` and longitude corresponds to `x`.

In [6]:
lat = centroid.y
lon = centroid.x
print('lat=%f, lon=%f' % (lat, lon))

lat=25.113527, lon=55.133696


Next, we get the timestamp of the image.

In [7]:
timestamp = dateutil.parser.parse(record.get('properties').get('timestamp'))

The timestamp is converted to the date time group format that the tide prediction algorithm expects (Y-m-d-H-M).

In [8]:
dtg = datetime.strftime(timestamp, '%Y-%m-%d-%H-%M')

In [9]:
print('dtg="%s"' % dtg)

dtg="2010-01-12-07-12"


At this point, we'd like to call the task with

In [10]:
print('lat="%s", lon="%s", dtg="%s"' % (str(lat), str(lon), dtg))

lat="25.1135269383", lon="55.1336961813", dtg="2010-01-12-07-12"


and receive an output like

```json
{
  "minimumTide24Hours":null,
  "maximumTide24Hours":null,
  "currentTide":null
}
```

We have currently implemented tide prediction using a modified version of bf-tideprediction (just without the flask wrapper for endpoints). The image `chambbj/hello-gbdx` v0.0.9 is available as GBDX task `hello-gbdx-chambbj`. It takes the aforementioned lat, lon, dtg all as strings.

In [339]:
# tide_task = gbdx.Task('bf-tideprediction-gbdx')
tide_task = gbdx.Task('hello-gbdx-chambbj')
tide_task.inputs.lat = str(lat)
tide_task.inputs.lon = str(lon)
tide_task.inputs.dtg = dtg

The tide prediction json file will be saved to `/mnt/work/output/json` and is persisted to `some_new_folder` within my user bucket/prefix on S3.

In [340]:
import uuid
from os.path import join
workflow = gbdx.Workflow([tide_task])
random_str = str(uuid.uuid4())
workflow.savedata(tide_task.outputs.json, location=join('some_random_folder', random_str))
workflow.execute()
workflow.status

{u'event': u'submitted', u'state': u'pending'}

Check periodically for status `complete`.

In [341]:
import time
while not workflow.status.get('state') == u'complete':
    time.sleep(1.0)
workflow.status

{u'event': u'succeeded', u'state': u'complete'}

And download the result. Yes, we could display it here, but this is good enough.

In [342]:
gbdx.s3.download(join('some_random_folder', random_str))

The tides.json file is now in the local directory.

In [306]:
print(join('some_random_folder', random_str))

some_random_folder/f0bf6dc6-9585-49c3-9344-976be4a43f02


In [343]:
workflow.events

[{u'event': u'submitted',
  u'note': u'',
  u'state': u'pending',
  u'task': u'hello-gbdx-chambbj_a55ea5b2',
  u'task_id': u'4623813900506692963',
  u'timestamp': u'2017-06-19T20:32:50.106084+00:00',
  u'when': u'3 minutes ago'},
 {u'event': u'scheduled',
  u'note': u'instance_id: i-c92fa74a',
  u'state': u'pending',
  u'task': u'hello-gbdx-chambbj_a55ea5b2',
  u'task_id': u'4623813900506692963',
  u'timestamp': u'2017-06-19T20:33:02.246718+00:00',
  u'when': u'2 minutes ago'},
 {u'event': u'started',
  u'note': u'instance_id: i-0c8535edb286159fa, domain: default',
  u'state': u'running',
  u'task': u'hello-gbdx-chambbj_a55ea5b2',
  u'task_id': u'4623813900506692963',
  u'timestamp': u'2017-06-19T20:33:02.754114+00:00',
  u'when': u'2 minutes ago'},
 {u'event': u'succeeded',
  u'note': u'instance_id: i-0c8535edb286159fa,  Note: ',
  u'state': u'complete',
  u'task': u'hello-gbdx-chambbj_a55ea5b2',
  u'task_id': u'4623813900506692963',
  u'timestamp': u'2017-06-19T20:34:43.574396+00:00'

In [344]:
workflow.stdout

[{'id': u'4623813900506692963',
  'name': u'hello-gbdx-chambbj_a55ea5b2',
  'stdout': u'<15>1 2017-06-19T20:34:28.665229Z 9dca1186b1e1 beachfront 16 - (beachfront.logger:init_logger) DEBUG Initialized logger\nusage: bfalg-ndwi [-h] -i INPUT [-b BANDS BANDS] [--outdir OUTDIR]\n                  [--basename BASENAME] [--l8bqa L8BQA] [--coastmask]\n                  [--minsize MINSIZE] [--close CLOSE] [--simple SIMPLE]\n                  [--smooth SMOOTH] [--verbose VERBOSE] [--version]\n\nBeachfront Algorithm: NDWI (v1.0.3)\n\noptional arguments:\n  -h, --help            show this help message and exit\n  -i INPUT, --input INPUT\n                        Input image (1 or 2 files) (default: None)\n  -b BANDS BANDS, --bands BANDS BANDS\n                        Band numbers for Green and NIR bands (default: [1, 1])\n  --outdir OUTDIR       Save intermediate files to this dir (otherwise temp)\n                        (default: )\n  --basename BASENAME   Basename to give to output files (no e

In [346]:
order_id = gbdx.ordering.order(cat_id)

In [348]:
gbdx.ordering.status(order_id)

[{u'acquisition_id': u'103001000349F800',
  u'location': u's3://receiving-dgcs-tdgplatform-com/056215432010_01_003',
  u'state': u'delivered'}]

In [349]:
gbdx.catalog.get_data_location(cat_id)

u's3://receiving-dgcs-tdgplatform-com/056215432010_01_003'

In [352]:
# multispectral 
aop1 = gbdx.Task('AOP_Strip_Processor',
                 data=gbdx.catalog.get_data_location(cat_id),
                 bands='MS',
                 enable_dra=False,
                 enable_pansharpen=False,
                 enable_acomp=True,
                 ortho_epsg='UTM')

wf = gbdx.Workflow([aop1])

random_str = str(uuid.uuid4())
wf.savedata(aop1.outputs.data, location=join('some_random_folder', random_str))
wf.execute()

u'4624261907042177904'

In [369]:
wf.status

{u'event': u'succeeded', u'state': u'complete'}

In [372]:
random_str

'69ca3c34-a4bb-4f9f-9e81-2bd839c3af34'